In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import truncnorm
import math
from tqdm.auto import tqdm
import sys
import os
import scipy
from scipy.stats import linregress
import statsmodels.api as sm
from scipy.optimize import minimize
import concurrent
from concurrent.futures import ProcessPoolExecutor
from joblib import Parallel, delayed
import time
import h5py

from Others import *

# Config

In [ ]:
ini = 0.5
config = {'N':400000,
    'real_A': 1, 'real_B': 1, 'real_C': 1, 'real_D': 1,
    'Q': 1, 'H': 1, 'x_0': 1, 'T': 1,
    'initial_A': ini, 'initial_B': ini, 
    'initial_C': ini, 'initial_D': ini, 
    'initial_nt': 100, 'initial_m_n':1, 
    'initial_v_n':1, 'initial_seed':None
}

In [ ]:
num_exp = 120
initial_seeds = list(range(1, num_exp + 1))

# Run Simulations

In [ ]:
def run_simulation_for_seed(config, initial_seed):
    try:
        local_config = config.copy()
        local_config['initial_seed'] = initial_seed
        simulator = Model_Based_LQ_Simulator(**local_config)
        simulator.run_many_iterations()
        all_regrets = simulator.get_all_regrets()
        file_name = f'Algo_seed_{initial_seed}.h5'
        
        with h5py.File('../Data_Model_Based/' + file_name, 'w') as f:
            f.create_dataset('phi_1_list', data=np.array(simulator.phi_1_list), compression="gzip")
            f.create_dataset('regrets', data=np.array(all_regrets), compression="gzip")
    except Exception as e:
        print(f"Failed to save file {file_name}: {e}")
        return initial_seed, [[], []]  # Handle the error gracefully
    
    return initial_seed, [simulator.phi_1_list, all_regrets]

def run_simulations_with_joblib(config, initial_seeds, num_workers=5):
    # Wrap the iterable with tqdm for progress indication
    results = Parallel(n_jobs=num_workers, backend="loky")(
        delayed(run_simulation_for_seed)(config, seed) for seed in tqdm(initial_seeds, total=len(initial_seeds))
    )
    res_dict = {seed: result for seed, result in results if result is not None}
    try:
        with h5py.File('../Data_Model_Based/Algo_full_results.h5', 'w') as f:
            for k, (phi_1_list, regrets) in res_dict.items():
                grp = f.create_group(str(k))
                grp.create_dataset('phi_1_list', data=np.array(phi_1_list), compression="gzip")
                grp.create_dataset('regrets', data=np.array(regrets), compression="gzip")
    except Exception as e:
        print(f"Failed to save collective results: {e}")

    return res_dict

In [ ]:
start_time = time.time()
results = run_simulations_with_joblib(config, initial_seeds, num_workers=15)
end_time = time.time()

elapsed_time = end_time - start_time
hours, remainder = divmod(elapsed_time, 3600)
minutes, seconds = divmod(remainder, 60)

print(f"Time taken for simulations with joblib: {int(hours)} hours, {int(minutes)} minutes, {seconds:.2f} seconds")